In [ ]:
import pandas as pd
import tweepy
import time
from datetime import datetime
import datetime
from datetime import datetime
import numpy as np
import os
import re
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame(columns=['username','tweet_date','retweet_count', 'tweet'])

In [ ]:
# Scraping might not be perfect...working to fix the issue....
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
username = input('enter for username (ex.@POTUS) or hashtag (ex.#covid19):')
i=0
for _ in range(1):
    dates = [datetime.datetime(2020, 5, 15, 0, 0, 0),datetime.datetime(2020, 5, 16, 0, 0, 0),datetime.datetime(2020, 5, 17, 0, 0, 0),]
    for starting_time in dates:
        for tweet in (tweepy.Cursor(api.search,q=username,count=100,
                                   lang="en",
                                   since=str(starting_time),
                                   until=str(starting_time+datetime.timedelta(days=1)),
                                  tweet_mode='extended',
                                  since_id=None).items(10000)):
            try:
                tweet_full_text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                tweet_full_text = tweet.full_text
            df = df.append({'username':tweet.user.screen_name,
                    'tweet_date': tweet.created_at,
                    'retweet_count': tweet.retweet_count,
                    'tweet':tweet_full_text},ignore_index=True)

            if (i+1)%1000==0:
                print('%d tweets downloaded'%(i+1))
            i+=1
        time.sleep(15*60)
to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
filename = to_csv_timestamp+'data.csv'
df.to_csv(filename,encoding='utf8')

In [ ]:
import pandas as pd
df = pd.read_csv('filename.csv',engine='python')
username = '#covid19'

In [ ]:
df = df.drop(columns=df.columns[0])

In [ ]:
dataset = df.copy()

In [ ]:
dataset.shape

In [ ]:
dataset.dropna(subset = ['tweet'], inplace = True)

In [ ]:
dataset.drop_duplicates(keep=False,inplace=True)

In [ ]:
data = dataset.copy()

In [ ]:
date = []
for i in range(0, len(data)):
    ith_date_str = str(data.iloc[i, :]['tweet_date'])
    ith_match = re.search(r'\d{4}-\d{2}-\d{2}', ith_date_str)
    ith_date = datetime.strptime(ith_match.group(), '%Y-%m-%d').date()
    date.append(ith_date)
data['dates'] = date

In [ ]:
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
def clean_text(x):
    x = re.sub('@[^\s]+', '',x)
    x = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', x)
    x = re.sub(r'#([^\s]+)', '', x)
    x = re.sub(r'^(?i)[RT]+','',x)
    x = emoji_pattern.sub(r'', x)
    x = x.lower()
    x = [i.strip(string.punctuation) for i in x.split(" ")]
    stopword = stopwords.words('english')
    x = [i for i in x if i not in stopword]
    x = [i for i in x if len(i)>1]
    pos_tags = pos_tag(x)
    x = [WordNetLemmatizer().lemmatize(i[0],get_wordnet_pos(i[1])) for i in pos_tags]
    x = " ".join(x)
    return(x)

In [ ]:
data['tweet_c'] = data.tweet.apply(lambda j: clean_text(j))

In [ ]:
data.reset_index(inplace=True,drop=True)

In [ ]:
sia = SentimentIntensityAnalyzer()
data['sentiment'] = data.tweet_c.apply(lambda x: sia.polarity_scores(x))
data = pd.concat([data.drop(['sentiment'], axis = 1), data['sentiment'].apply(pd.Series)], axis = 1)

In [ ]:
sentimentclass_list = []
for i in range(0, len(data)):
    curr_compound = data.compound.iloc[i]
    if (curr_compound <= 1.0 and curr_compound >= 0.55):
        sentimentclass_list.append(5)
    elif (curr_compound < 0.55 and curr_compound >= 0.10):
        sentimentclass_list.append(4)
    elif (curr_compound < 0.10 and curr_compound > -0.10):
        sentimentclass_list.append(3)
    elif (curr_compound <= -0.10 and curr_compound > -0.55):
        sentimentclass_list.append(2)
    elif (curr_compound <= -0.55 and curr_compound >= -1.00):
        sentimentclass_list.append(1)
data['sentiment_class'] = sentimentclass_list

In [ ]:
very_pos_text=""
very_neg_text=""
neut_text=""
pos_text=""
neg_text=""
for i in range(len(data)):
    if (data.iloc[i,:]['sentiment_class'])==5:
        very_pos_text+=data.iloc[i,:]['tweet_c']
    elif (data.iloc[i,:]['sentiment_class'])==4:
        pos_text+=data.iloc[i,:]['tweet_c']
    elif (data.iloc[i,:]['sentiment_class'])==2:
        neg_text+=data.iloc[i,:]['tweet_c']
    elif (data.iloc[i,:]['sentiment_class'])==1:
        very_neg_text+=data.iloc[i,:]['tweet_c']
    else:
        neut_text+=data.iloc[i,:]['tweet_c']
list_text = {'Positive':pos_text,
             'Negative':neg_text,
             'Netural':neut_text,
            'Very Positive':very_pos_text,
            'Very Negative':very_neg_text}
for text in list_text:
    if len(list_text[text])>0:
        wordcloud = WordCloud(
        background_color = 'black',
        max_words = 100,
        max_font_size = 40, 
        scale = 3,
        random_state = 42
    ).generate(list_text[text])
        plt.figure(1, figsize = (20, 15))
        plt.axis('off')
        plt.title(text, fontsize = 20)
        plt.imshow(wordcloud)
        plt.show()

In [ ]:
if username[0]=='#':
    data.tweet = data.tweet.str.replace(username,'',regex=True,flags=re.IGNORECASE)
else:
    data.tweet = data.tweet.str.replace(username,'',regex=True,flags=re.IGNORECASE)

In [ ]:
hashtaglist_d,mentionslist_d = [],[]
hashtaglist_d.append(data.tweet.apply(lambda x: list(set(re.findall(r'#[A-Z0-9a-z:]+',x)))))
mentionslist_d.append(data.tweet.apply(lambda x: list(set(re.findall(r'@[A-Z0-9a-z:]+',x)))))
hashtaglist,mentionslist = [],[]
for i in hashtaglist_d[0]:
    for j in i:
        hashtaglist.append(j)
for i in mentionslist_d[0]:
    for j in i:
        mentionslist.append(j)
df_hashtag = pd.DataFrame({'hashtags':hashtaglist})
df_mentions_on_user = pd.DataFrame({'Mentions':mentionslist})
top_hashtags = df_hashtag.groupby(['hashtags']).size().reset_index(name = 'counts').sort_values(by = 'counts', ascending = False).head(15)
top_mentions = df_mentions_on_user.groupby(['Mentions']).size().reset_index(name = 'counts').sort_values(by = 'counts', ascending = False).head(15)

In [ ]:
data.retweet_count = pd.to_numeric(data.retweet_count)

In [ ]:
top_tweets_df = data.sort_values(by = ['retweet_count'], ascending = False)
top_tweets_df.drop_duplicates(subset='tweet',keep='first',inplace=True)

In [ ]:
timeseries_hashtags = pd.DataFrame(columns = ['hashtags', 'count', 'date', 'dayofnov'])
mentionseries_hashtags = pd.DataFrame(columns = ['mentions', 'count', 'date', 'dayofnov'])
unique_date = data.dates.unique()

In [ ]:
def visualizing(main_df, timeseries_df,mentionseries_df, N, T, unique_dates):
    counter = 1
    for ith_date in reversed(unique_dates):
        if counter <= T:
            ith_date_df = main_df[main_df['dates'] == ith_date]
            ith_hashtag_list = []
            ith_mention_list = []
            hashtaglist_d,mentionslist_d = [],[]
            hashtaglist_d.append(ith_date_df.tweet.apply(lambda x: list(set(re.findall(r'#[A-Z0-9a-z:]+',x)))))
            mentionslist_d.append(ith_date_df.tweet.apply(lambda x: list(set(re.findall(r'@[A-Z0-9a-z:]+',x)))))
            for i in hashtaglist_d[0]:
                for j in i:
                    ith_hashtag_list.append(j)
            for i in mentionslist_d[0]:
                for j in i:
                    ith_mention_list.append(j)
            ith_df_hashtag = pd.DataFrame({'hashtags': ith_hashtag_list})
            ith_df_mention = pd.DataFrame({'mentions': ith_mention_list})
            ith_top_hashtags = ith_df_hashtag.groupby(['hashtags']).size().reset_index(name = 'count').sort_values(by = 'count', ascending = False).head(N)
            ith_top_mentions = ith_df_mention.groupby(['mentions']).size().reset_index(name = 'count').sort_values(by = 'count', ascending = False).head(N)
            ith_top_hashtags['date'] = ith_date
            ith_top_hashtags['dayofnov'] = ith_date.day
            ith_top_mentions['date'] = ith_date
            ith_top_mentions['dayofnov'] = ith_date.day
            timeseries_df = pd.concat([timeseries_df, ith_top_hashtags], axis = 0)
            mentionseries_df = pd.concat([mentionseries_df, ith_top_mentions], axis = 0)
            counter += 1
        else: 
            break
    timeseries_df.reset_index(inplace = True, drop = True)
    mentionseries_df.reset_index(inplace = True, drop = True)
    return timeseries_df,mentionseries_df

In [ ]:
timeseries_hashtags,mentionseries_hashtags = visualizing(main_df = data,
                       timeseries_df = timeseries_hashtags,
                        mentionseries_df = mentionseries_hashtags,
                       N = 15,
                       T = 4,
                       unique_dates = unique_date)

In [ ]:
fig = make_subplots(
    rows = 6, cols = 12,
    specs = [
        [None,None,None,{"type": "bar", "colspan":4},None,None,{"type": "bar", "colspan":4},None,None,None,None,None],
        [{'type':'pie','rowspan':3,'colspan':3},None,None,{"type": "bar", "colspan":4},None,None,None,None,{"type": "bar", "colspan":4},None,None,None],
        [None,None,None,{"type": "bar", "colspan":4},None,None,None,None,{"type": "bar", "colspan":4},None,None,None],
        [None,None,None,None,None,None,None,None,None,None,None,None],
        [{'type':'table','colspan':3},None,None,None,{'type':'table','colspan':3},None,None,None,{'type':'table','colspan':3},None,None,None],
         [None,None,{'type':'table','colspan':3},None,None,None,{'type':'table','colspan':3},None,None,None,None,None]
    ])

In [ ]:

fig.add_trace(go.Bar(
                x = top_hashtags.hashtags.tolist(),
                y = top_hashtags.counts.tolist(),
                name = 'Top hashtags',
                showlegend = True),
             row = 2 ,col = 4 )
fig.add_trace(go.Bar(
                x = top_mentions.Mentions.tolist(),
                y = top_mentions.counts.tolist(),
                name = 'Top Mentions',
                showlegend = True),
             row = 3 ,col = 4 )
for i in np.unique(timeseries_hashtags.date):
    dum = timeseries_hashtags[timeseries_hashtags.date==i]
    fig.add_trace(go.Bar(
                x = dum.hashtags.tolist(),
                y = dum['count'].tolist(),
                name = 'Top hashtags in'+'\t'+str(i),
                showlegend = True),
             row = 2 ,col = 9)
for i in np.unique(mentionseries_hashtags.date):
    dum = mentionseries_hashtags[mentionseries_hashtags.date==i]
    fig.add_trace(go.Bar(
                x = dum.mentions.tolist(),
                y = dum['count'].tolist(),
                name = 'Top Mentions in'+'\t'+str(i),
                showlegend = True),
             row = 3 ,col = 9)
fig.add_trace(go.Pie(labels=data.sentiment_class.replace({5:'Very Positive',4:'Positive',3:'Neutral',2:'Negative',1:'Very Negative'}).unique().tolist(),
       values = data.sentiment_class.replace({5:'Very Positive',4:'Positive',3:'Neutral',2:'Negative',1:'Very Negative'}).value_counts().tolist(),
                        textposition = "inside",showlegend=False,
                         texttemplate='%{label}: %{value:} <br>(%{percent})',
                        hoverinfo='none',
                        hole =0.3,
                         insidetextorientation='radial',
                    title={'text':'Sentiment Analysis','position':'middle center'}),row=2,col = 1)
fig.add_trace( go.Table(
    header = dict(values = list(['Positive Tweet']),
                 fill_color = 'blue',
                 align='center'),
    cells = dict(values=[data[data.sentiment_class==4].tweet.iloc[np.random.randint(0,len(data[data.sentiment_class==4]),5)]],
                align = 'left')),row = 5,col = 1
)
fig.add_trace( go.Table(
    header = dict(values = list(['Very Positive tweet']),
                 fill_color = 'orange',
                 align='center'),
    cells = dict(values=[data[data.sentiment_class==5].tweet.iloc[np.random.randint(0,len(data[data.sentiment_class==5]),5)]],
                align = 'left')),row = 6,col = 3
)
fig.add_trace( go.Table(
    header = dict(values = list(['Neutral tweet']),
                 fill_color = 'purple',
                 align='center'),
    cells = dict(values=[data[data.sentiment_class==3].tweet.iloc[np.random.randint(0,len(data[data.sentiment_class==3]),5)]],
                align = 'left')),row = 5,col = 9
)
fig.add_trace( go.Table(
    header = dict(values = list(['Negative tweet']),
                 fill_color = 'green',
                 align='center'),
    cells = dict(values=[data[data.sentiment_class==2].tweet.iloc[np.random.randint(0,len(data[data.sentiment_class==2]),5)]],
                align = 'left')),row = 5,col = 5
)
fig.add_trace( go.Table(
    header = dict(values = list(['Very Negative tweet']),
                 fill_color = 'red',
                 align='center'),
    cells = dict(values=[data[data.sentiment_class==1].tweet.iloc[np.random.randint(0,len(data[data.sentiment_class==1]),5)]],
                align = 'left')),row = 6,col = 7
)

fig.update_layout(
    template="plotly_dark",
    title = {'text':"Twitter Analysis for "+username,'x':0.5,'y':0.9,'xanchor':'center'},
    titlefont = {'size':40},
    
    showlegend=True,
    legend_orientation="v" 
)

In [ ]:
fig.write_html('Twitter.html', auto_open=True)